<a href="https://colab.research.google.com/github/NLarchive/AI/blob/main/notebookchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTODEBUG

In [3]:
# --- Cell 1: API Setup and Authentication ---
from google.colab import userdata
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time

gemini_api_secret_name = 'GOOGLE_API_KEY'

try:
    GOOGLE_API_KEY = userdata.get(gemini_api_secret_name)
    genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
    print(f'''Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}''')
    raise e
except userdata.NotebookAccessError as e:
    print(
        f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
    raise e
except Exception as e:
    print(
        f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
    raise e

llm = genai.GenerativeModel('models/gemini-1.5-flash-001')

# --- Cell 2: Context Caching Setup ---
def setup_cache(system_instruction, contents, ttl_minutes):
    if len(contents) < 32768:  # Ensure the minimum token requirement for caching
        print("Caching skipped: content does not meet the minimum token requirement.")
        return None
    cache = caching.CachedContent.create(
        model='models/gemini-1.5-flash-001',
        display_name='notebook_context_cache',
        system_instruction=system_instruction,
        contents=contents,
        ttl=datetime.timedelta(minutes=ttl_minutes),
    )
    return cache

# --- Cell 3: LLM Interaction Class ---
class LLM:
    def __init__(self, cache=None):
        self.model = genai.GenerativeModel.from_cached_content(cache) if cache else genai.GenerativeModel('models/gemini-1.5-flash-001')
        self.chat = None

    def chat_with_llm(self, message: str) -> str:
        if self.chat is None:
            self.chat = self.model.start_chat(history=[])

        response = self.chat.send_message(message)
        return response.text

# --- Cell 4: Notebook Debugger Class ---
import nbformat

class NotebookDebugger:
    def __init__(self, llm):
        self.llm = llm

    def analyze_code(self, code):
        prompt = f"You are an expert Python debugger. Analyze the following code for potential issues, inefficiencies, or improvements:\n\n{code}"
        analysis = self.llm.chat_with_llm(prompt)
        return analysis

    def suggest_improvements(self, code):
        prompt = f"You are an expert Python developer. Suggest improvements for the following code, focusing on efficiency, readability, and best practices:\n\n{code}"
        suggestions = self.llm.chat_with_llm(prompt)
        return suggestions

# --- Cell 5: Notebook Chat Function ---
def chat_with_notebook(file_path: str, cache=None):
    llm_instance = LLM(cache)
    debugger = NotebookDebugger(llm_instance)

    try:
        with open(file_path, 'r') as f:
            notebook = nbformat.read(f, as_version=4)
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid Jupyter notebook.")
        return

    print("Notebook loaded. You can now debug and improve the code in this notebook.")

    while True:
        print("\nOptions:")
        print("1. Analyze a specific cell")
        print("2. Suggest improvements for a specific cell")
        print("3. Analyze entire notebook")
        print("4. Chat with LLM about the notebook")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice in ('1', '2'):
            try:
                cell_num = int(input("Enter the cell number to analyze/improve: ")) - 1
                if 0 <= cell_num < len(notebook.cells):
                    code = notebook.cells[cell_num].source
                    if choice == '1':
                        result = debugger.analyze_code(code)
                        print("\nAnalysis result:")
                    else:
                        result = debugger.suggest_improvements(code)
                        print("\nSuggested improvements:")
                    print(result)
                else:
                    print("Invalid cell number.")
            except ValueError:
                print("Please enter a valid cell number.")

        elif choice == '3':
            for i, cell in enumerate(notebook.cells):
                if cell.cell_type == 'code':
                    print(f"\nAnalyzing Cell {i + 1}:")
                    result = debugger.analyze_code(cell.source)
                    print(result)

        elif choice == '4':
            while True:
                query = input("Enter your question about the notebook (or type 'back' to return to main menu): ")
                if query.lower() == 'back':
                    break
                prompt = f"Here is a Jupyter Notebook with various code cells:\n\n"
                for cell in notebook.cells:
                    if cell.cell_type == 'code':
                        prompt += f"Cell {cell.execution_count if hasattr(cell, 'execution_count') else 'N/A'}:\n{cell.source}\n\n"

                prompt += f"\nUser's question: {query}\n\nProvide insights or answers based on the notebook."

                try:
                    llm_response = llm_instance.chat_with_llm(prompt)
                    print("\nResponse:")
                    print(llm_response)
                    print("-" * 50)
                except Exception as e:
                    print(f"Error during LLM interaction: {str(e)}")

        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# --- Cell 6: Main Execution Block ---
from google.colab import drive
import json
if __name__ == "__main__":
    drive.mount('/content/drive')
    notebook_path = '/content/drive/MyDrive/AI/AI_modules/notebookchat.ipynb'  # Replace with your notebook path

    # Setup cache with system instructions
    system_instruction = (
        "You are an AI assistant for debugging and improving Jupyter Notebooks. Use best practices to provide insights."
    )
    cache = None
    if len([]) >= 32768:  # Check if contents exceed token minimum for caching
        cache = setup_cache(system_instruction, [], ttl_minutes=60)  # Cache with a 1-hour TTL

    chat_with_notebook(notebook_path, cache)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Notebook loaded. You can now debug and improve the code in this notebook.

Options:
1. Analyze a specific cell
2. Suggest improvements for a specific cell
3. Analyze entire notebook
4. Chat with LLM about the notebook
5. Exit
Enter your choice (1-5): 3

Analyzing Cell 2:
## Analysis of the Python Code for Debugging and Improving Jupyter Notebooks

This code provides a framework for interacting with a Jupyter Notebook through a chat interface, utilizing a large language model (LLM) for analysis and improvement suggestions. Here's a breakdown of potential issues, inefficiencies, and improvements:

**1. API Setup and Authentication:**

* **Potential Issues:**
    * **Secret Storage:** Using `userdata.get()` for API key retrieval makes the code dependent on Colab environment and requires manual secret creation. It's not portable to other environments.
    * **Har

KeyboardInterrupt: Interrupted by user